In [3]:
# 6. 실행 파일 1 (최적화 비중 뽑아내는 것 까지)

import pandas as pd
import numpy as np
import os
import config
from data_manager import DataManager
from portfolio_optimizer import PortfolioOptimizer

# 1. 데이터 가져오기
dm = DataManager()
stock_data = dm.get_data(config.STOCKS, config.START_DATE, config.END_DATE, save_path=config.RAW_DATA_PATH)

# 2. 최적화 실행
if not stock_data.empty:
    results = {}
    final_tickers = stock_data.columns.tolist()
    num_assets = len(final_tickers)
    
    # 공통 최적화 설정
    init_guess = np.array([1/num_assets] * num_assets)
    # ✨ config 파일에서 범용 비중 제약조건 불러오기 ✨
    bounds = tuple(((config.WEIGHT_CONSTRAINTS['min'], config.WEIGHT_CONSTRAINTS['max'])) for _ in range(num_assets))
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
    
    daily_returns = dm.calculate_returns(stock_data, 'daily')
    optimizer = PortfolioOptimizer(daily_returns)

    # config.py에서 실행할 전략 목록 불러오기
    for name, params in config.STRATEGIES.items():
        if not params.get('enabled', False):
            continue
            
        print(f"\n--- '{name}' 모델 분석 시작 ---")
        
        if params.get('is_optimizer') is False:
            weights = optimizer.get_risk_parity_weights(daily_returns)
        else:
            current_constraints = list(constraints)
            target_return = params.get('target_return')
            
            if target_return is not None:
                current_constraints.append({'type': 'ineq', 'fun': lambda w: optimizer.calculate_metrics(w)['return'] - target_return})

            # ✨ 모든 최적화 전략에 범용 비중 제약(bounds) 적용 ✨
            weights = optimizer.run_optimization(params['objective'], constraints=current_constraints, bounds=bounds)
        
        results[name] = {'weights': weights}
        print(f"최적 가중치: " + ", ".join([f"{s}: {w*100:.2f}%" for s, w in zip(final_tickers, weights)]))

    # 3. 결과 저장
    if results:
        os.makedirs(config.RESULTS_DIR, exist_ok=True)
        with pd.ExcelWriter(config.OPTIMAL_WEIGHTS_SAVE_PATH, engine='openpyxl') as writer:
            for model_name, data in results.items():
                weights_df = pd.DataFrame({'Ticker': final_tickers, 'Optimal_Weight': data['weights']})
                weights_df.to_excel(writer, sheet_name=f'{model_name}_weights', index=False)
        print(f"\n모든 모델의 최적 가중치가 저장되었습니다: {config.OPTIMAL_WEIGHTS_SAVE_PATH}")


'['SPY', 'QQQ', 'IWM', 'VTI', 'AGG', 'XLK', 'XLV', 'XLF', 'XLE', 'XLP', 'VIG', 'GLD', 'BTC-USD', 'ETH-USD', 'SOL-USD', 'XRP-USD', 'ADA-USD', 'AVAX-USD', 'LINK-USD', 'DOT-USD']' 종목 데이터 다운로드 시도 중...


[*********************100%***********************]  20 of 20 completed


수정 종가 데이터가 로컬에 저장되었습니다: /Users/012oov/Documents/Quant/MPT_Back/Data/Raw data/downloaded_stock_prices.csv

--- 'max_calmar' 모델 분석 시작 ---


/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


최적 가중치: SPY: 1.00%, QQQ: 1.00%, IWM: 1.00%, VTI: 1.00%, AGG: 1.00%, XLK: 1.00%, XLV: 1.00%, XLF: 15.63%, XLE: 26.12%, XLP: 1.00%, VIG: 1.00%, GLD: 30.00%, BTC-USD: 3.79%, ETH-USD: 1.00%, SOL-USD: 7.71%, XRP-USD: 2.74%, ADA-USD: 1.00%, AVAX-USD: 1.00%, LINK-USD: 1.00%, DOT-USD: 1.00%

--- 'max_sharpe' 모델 분석 시작 ---


/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


최적 가중치: SPY: 1.00%, QQQ: 1.00%, IWM: 1.00%, VTI: 1.00%, AGG: 1.00%, XLK: 1.00%, XLV: 1.00%, XLF: 15.63%, XLE: 26.12%, XLP: 1.00%, VIG: 1.00%, GLD: 30.00%, BTC-USD: 3.79%, ETH-USD: 1.00%, SOL-USD: 7.71%, XRP-USD: 2.74%, ADA-USD: 1.00%, AVAX-USD: 1.00%, LINK-USD: 1.00%, DOT-USD: 1.00%

--- 'risk_parity' 모델 분석 시작 ---
최적 가중치: SPY: 6.70%, QQQ: 5.06%, IWM: 5.07%, VTI: 6.56%, AGG: 19.23%, XLK: 4.69%, XLV: 8.08%, XLF: 5.94%, XLE: 4.03%, XLP: 8.92%, VIG: 7.97%, GLD: 7.78%, BTC-USD: 1.93%, ETH-USD: 1.45%, SOL-USD: 0.99%, XRP-USD: 1.04%, ADA-USD: 1.20%, AVAX-USD: 0.97%, LINK-USD: 1.18%, DOT-USD: 1.21%

--- 'min_variance' 모델 분석 시작 ---
최적 가중치: SPY: 1.00%, QQQ: 1.00%, IWM: 1.00%, VTI: 1.00%, AGG: 30.00%, XLK: 1.00%, XLV: 3.46%, XLF: 1.00%, XLE: 1.00%, XLP: 24.75%, VIG: 1.00%, GLD: 25.79%, BTC-USD: 1.00%, ETH-USD: 1.00%, SOL-USD: 1.00%, XRP-USD: 1.00%, ADA-USD: 1.00%, AVAX-USD: 1.00%, LINK-USD: 1.00%, DOT-USD: 1.00%

--- 'target_return' 모델 분석 시작 ---


/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


최적 가중치: SPY: 1.00%, QQQ: 1.00%, IWM: 1.00%, VTI: 1.00%, AGG: 26.68%, XLK: 1.00%, XLV: 1.00%, XLF: 7.27%, XLE: 10.94%, XLP: 9.60%, VIG: 1.00%, GLD: 30.00%, BTC-USD: 1.00%, ETH-USD: 1.00%, SOL-USD: 1.52%, XRP-USD: 1.00%, ADA-USD: 1.00%, AVAX-USD: 1.00%, LINK-USD: 1.00%, DOT-USD: 1.00%

모든 모델의 최적 가중치가 저장되었습니다: /Users/012oov/Documents/Quant/MPT_Back/Results/optimized_weights_all_periods.xlsx
